In [1]:
import numpy as np 
import cv2
import imutils
cap = cv2.VideoCapture("/home/roar/Desktop/LaneDetection/training_1000.MOV")
cap.isOpened()

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55ebe7898a80] Referenced QT chapter track not found


True

In [2]:
def floodfilled(frame):
    img = frame.copy()
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    img_hsv = cv2.GaussianBlur(img_hsv, (5,5), 0)

    #frame = imutils.resize(frame,width = 960)
    seed = (1079, 920)

    box = (200, 200)
    x = (seed[0] - box[0] - box[1], seed[0])
    y = (seed[1] - box[1], seed[1] + box[1])


    mean = np.mean(img_hsv[x[0]:x[1], y[0]:y[1], :], axis = (0,1))
    # std = np.std(frame[x[0]:x[1], y[0]:y[1], :], axis = (0,1))
    # scale = 2.5
    # scale_std = std * scale
    # scale_std[2] = 200
    # scale_std[1] = 50
    # print(scale_std)
    # break
    scale_std = [50, 50, 200]
    # print(mean, '/n',std)

    img_hsv[seed[0], seed[1]] = mean

    #crop_frame = frame[200:]
    #print(crop_frame.shape)

    
    mask = np.zeros((img_hsv.shape[0] + 2, img_hsv.shape[1] + 2)).astype(np.uint8)
    cv2.floodFill(img_hsv, mask, seedPoint=seed, newVal=(255, 0, 0), \
        loDiff=tuple(scale_std), upDiff=tuple(scale_std), flags= cv2.FLOODFILL_FIXED_RANGE)
    cv2.circle(img_hsv, seed, 2, (0, 255, 0), cv2.FILLED, cv2.LINE_AA)
    mask = mask * 255
    return img_hsv, mask


In [3]:
def grass_detection(frame, remove_niose : bool = False):
    
    img = frame.copy()
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    kernel_size = 5
    blur = cv2.GaussianBlur(img_hsv, (5,5), 0)
    
    low_green = np.array([25, 52, 72])
    high_green = np.array([200, 200, 255])

    filter_size = (150, img_hsv.shape[1])
    sky_filter = np.zeros(filter_size)

    flt = cv2.inRange(blur[150:], low_green, high_green)
    combo_filter = np.concatenate((sky_filter, flt), axis = 0)
    kernel = np.ones((kernel_size*2,kernel_size*2),np.uint8)
    dilation_filter = cv2.dilate(combo_filter, kernel, iterations=1)
    mask = cv2.morphologyEx(combo_filter, cv2.MORPH_OPEN, None).astype(np.uint8)

    return mask

In [4]:
def hsv_road_detection(frame):

    img = frame.copy()
    
    lower_road = np.array([117, 0, 98])
    upper_road = np.array([179, 20, 204])
    
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Threshold the HSV image to get only road colors
    mask = cv2.inRange(hsv_img, lower_road, upper_road)

    mask = cv2.erode(mask, np.ones((3, 3), np.uint8))
    mask = cv2.dilate(mask, np.ones((5, 5), np.uint8))

    return mask

In [5]:
GROUND_MASK_LOWER = np.array([0,0,80],dtype='uint8')
GROUND_MASK_UPPER = np.array([255,50,200],dtype='uint8')
GRASS_MASK_LOWER = np.array([43,50,20],dtype='uint8')
GRASS_MASK_UPPER = np.array([128,255,255],dtype='uint8')

dilkernel = np.ones((5,5), np.uint8)
erokernel = np.ones((6,6), np.uint8)
erokernel[:2,:] = 0
erokernel[-2:,:] = 0
erokernel[:,:2]=0
erokernel[:,-2:]=0

def groundAndGrassMask(image):
    hsvImg = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    ground = cv2.inRange(hsvImg,GROUND_MASK_LOWER,GROUND_MASK_UPPER)
    grass = cv2.inRange(hsvImg,GRASS_MASK_LOWER,GRASS_MASK_UPPER)
    
    grass = cv2.erode(grass,erokernel,iterations=2)
    ground = cv2.erode(ground,erokernel,iterations=2)
    ground = cv2.dilate(ground,dilkernel,iterations=10)
    grass = cv2.dilate(grass, dilkernel, iterations=10)
    
    combined = cv2.bitwise_and(ground,cv2.bitwise_not(grass))
    combined = cv2.erode(combined,erokernel,iterations=4)
    #combined = cv.dilate(combined,dilkernel,iterations=5)
    
    return ground,grass,combined

def getContour(alpha):
    #https://stackoverflow.com/questions/66753026/opencv-smoother-contour
    contours,hierachy = cv2.findContours(alpha,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    big_contour = max(contours, key=cv2.contourArea)
    contour_img = np.zeros_like(alpha)
    cv2.drawContours(contour_img, [big_contour], 0, 255, -1)
    # apply dilate to connect the white areas in the alpha channel
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (40,40))
    dilate = cv2.morphologyEx(contour_img, cv2.MORPH_DILATE, kernel)
    return dilate, None

def road_and_grass_detection(frame):
    img = frame.copy()
    ground, grass, comb = groundAndGrassMask(img)
    dilatedContour, edge = getContour(comb)
    return dilatedContour

In [6]:
def edge_mask(mask):
    mask_copy = mask.copy()
    gx, gy = np.gradient(mask_copy)
    temp_edge = gy * gy + gx * gx
    temp_edge[temp_edge != 0.0] = 255.0
    temp_edge = temp_edge.astype(np.uint8)

    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(temp_edge, connectivity=8)
    sizes = stats[1:, -1]
    nb_components = nb_components - 1
    min_size = 1000  #num pixels

    img2 = np.zeros((output.shape))
    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            img2[output == i + 1] = 255

    return img2

def closest_edge(mask_of_edge):
    final_mask = np.zeros_like(mask_of_edge)
    for j in range(mask_of_edge.shape[1]):
        # ys = np.where(img2[:, j] !=0 )
        # print(ys)
        # if ys:
        #     print("empty")
        #     continue
        # else:
        #     print("not empty")
        #     i = np.max(ys)
        #     img3 = [i][j] = 255
        for i in reversed(range(mask_of_edge.shape[0])):
            if mask_of_edge[i][j] != 0:
                final_mask[i][j] = 255
                break
    return final_mask


    

In [9]:
NUM_FRAMES_PER_TIME = 20
frameCount = 0
while True:
    ret,frame = cap.read()
    if not ret:
        cv2.destroyAllWindows()
        break

    frameCount += NUM_FRAMES_PER_TIME # i.e. at 30 fps, this advances one second
    cap.set(cv2.CAP_PROP_POS_FRAMES, frameCount)

    # cv2.namedWindow("original", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow("floodfilled road", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow("floodfilled mask", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow("grass detection mask", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow("road detection mask", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow("road & grass detection mask", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow('and mask', cv2.WINDOW_KEEPRATIO)

    # cv2.namedWindow("floodfilled lane", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow("grass detection lane", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow("road detection lane", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow("road & grass detection lane", cv2.WINDOW_KEEPRATIO)
    # cv2.namedWindow('and lane', cv2.WINDOW_KEEPRATIO)


    # floodfilled_road, floodfilled_mask = floodfilled(frame)
    grass_detection_mask = grass_detection(frame)
    road_detection_mask = hsv_road_detection(frame)
    # rg_detection_mask = road_and_grass_detection(frame)
    and_mask = np.bitwise_and(road_detection_mask, np.bitwise_not(grass_detection_mask))

    # floodfilled_edge = edge_mask(floodfilled_mask)
    grass_edge = edge_mask(grass_detection_mask)
    road_edge = edge_mask(road_detection_mask)
    # rg_edge = edge_mask(rg_detection_mask)
    and_edge = edge_mask(and_mask)

    # cv2.imshow('original', frame)
    # cv2.imshow('floodfilled road', floodfilled_road)
    # cv2.imshow('floodfilled mask', floodfilled_mask)
    cv2.imshow('grass detection mask', grass_detection_mask)
    cv2.imshow('road detection mask', road_detection_mask)
    # cv2.imshow('road & grass detection mask', rg_detection_mask)
    cv2.imshow('and mask', and_mask)

    # cv2.imshow('floodfilled lane', floodfilled_edge)
    # cv2.imshow('grass detection lane', grass_edge)
    # cv2.imshow('road detection lane', road_edge)
    # cv2.imshow('road & grass detection lane', rg_edge)
    cv2.imshow('and lane', and_edge)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break




In [ ]:
_